<a href="https://colab.research.google.com/github/sinajahangir/Cload-Data-Retrieval/blob/main/NWMRetrieval_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First version: May 2025
Sina Jahangir

Downloads and processes NWM (National Water Model) NetCDF data stored on AWS S3.

Extracts time-series data for specific (x, y) locations efficiently using Dask chunking for large datasets.

Saves results to a Pandas DataFrame for analysis or plotting.

Key Features:

✅ Lazy Loading with Dask – Handles large datasets without crashing Colab's RAM.

✅ S3 Integration – Directly reads NetCDF files from AWS S3.

✅ Nearest-Point Extraction – Uses xarray.sel() with method="nearest" for location-based queries.

✅ Parallel Processing – Optional parallel=True for faster multi-file reads.

MIT License
Copyright (c) [2025] [Sina Jahangir]

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

#Install dependencies

In [ ]:
!pip install s3fs #install s3fs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 115.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.5.1 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cup

In [ ]:
import xarray as xr
import pandas as pd
import s3fs # For accessing S3
import pyproj
import numpy as np # For handling numpy types from attributes

#Access S3 Amazon bucket for data retrieval

In [ ]:
S3_BUCKET_NAME = "noaa-nwm-retrospective-2-1-pds"
NWM_VERSION_PATH = "model_output" # For v2.1
PRODUCT_TYPE = "LDASOUT"          # LDASOUT
'''
1. CHRTOUT: Every hour, channel network output
2. LAKEOUT: Every hour, reservoir (lake) output
3. GWOUT: Every hour, conceptual groundwater output
4. LDASOUT: Every 3 hours, land model output
5. RTOUT: Every 3 hours, high-resolution terrain routing output
Model resolution: 1-km land surface grid; 250-m terrain routing grid
'''

'\n1. CHRTOUT: Every hour, channel network output\n2. LAKEOUT: Every hour, reservoir (lake) output\n3. GWOUT: Every hour, conceptual groundwater output\n4. LDASOUT: Every 3 hours, land model output\n5. RTOUT: Every 3 hours, high-resolution terrain routing output\nModel resolution: 1-km land surface grid; 250-m terrain routing grid\n'

# Define period and variables of ineterest

In [ ]:
# Define your time range
END_YEAR = "2003"
START_YEAR='2003'
START_DATETIME_STR = f"{START_YEAR}-01-01T00:00:00"
END_DATETIME_STR = f"{END_YEAR}-01-02T00:00:00" # Example: LDASOUT has 3-hour cycles

In [ ]:
# Target Location (Latitude and Longitude); Change based on the desired output
TARGET_LAT = 44.762
TARGET_LON = -90.1111


# --- 2. Generate File URLs ---
print(f"Generating file URLs for {START_DATETIME_STR} to {END_DATETIME_STR} for {PRODUCT_TYPE}...")
# LDASOUT is every 3rd hour
datetime_range = pd.date_range(start=START_DATETIME_STR, end=END_DATETIME_STR, freq='3h')

Generating file URLs for 2003-01-01T00:00:00 to 2003-01-02T00:00:00 for LDASOUT...


In [ ]:
#Generate urls. These will be used to access the data
s3_file_urls = []
for dt in datetime_range:
    year_str = dt.strftime('%Y')
    filename = f"{dt.strftime('%Y%m%d%H')}00.{PRODUCT_TYPE}_DOMAIN1.comp"
    s3_path = f"s3://{S3_BUCKET_NAME}/{NWM_VERSION_PATH}/{year_str}/{filename}"
    s3_file_urls.append(s3_path)

if not s3_file_urls:
    print("No file URLs generated. Check your date range and product type.")
    exit()
#201701010000.LDASOUT_DOMAIN1.comp
print(f"Generated {len(s3_file_urls)} S3 file URLs. Example: {s3_file_urls[-1]}")

Generated 9 S3 file URLs. Example: s3://noaa-nwm-retrospective-2-1-pds/model_output/2003/200301020000.LDASOUT_DOMAIN1.comp


# Connecting to S3 and troubleshooting

In [ ]:
s3 = s3fs.S3FileSystem(anon=True) #Connect to S3

In [ ]:
# Coordinate names in the LDASOUT files for the projected grid
X_COORD_NAME = 'x'
Y_COORD_NAME = 'y'
CRS_VARIABLE_NAME = 'crs' # The variable holding projection info

In [ ]:
def inspect_sample_file(file_url, s3_fs):
    print(f"\n--- Inspecting sample file: {file_url} ---")
    try:
        with s3_fs.open(file_url, 'rb') as f_obj:
            ds_sample = xr.open_dataset(f_obj, engine='h5netcdf')
            print("Sample file opened successfully.")
            print("\nVariables:")
            for var_name in ds_sample.variables: print(f"- {var_name} (dims: {ds_sample[var_name].dims}, attrs: {ds_sample[var_name].attrs})")
            print("\nCoordinates:")
            for coord_name in ds_sample.coords: print(f"- {coord_name} (dims: {ds_sample[coord_name].dims}, attrs: {ds_sample[coord_name].attrs})")
            print("\nDimensions:", ds_sample.dims)
            if CRS_VARIABLE_NAME in ds_sample:
                print(f"\nAttributes of '{CRS_VARIABLE_NAME}' variable:")
                for attr_name, attr_val in ds_sample[CRS_VARIABLE_NAME].attrs.items():
                    print(f"  - {attr_name}: {attr_val}")
            else:
                print(f"\nWarning: CRS variable '{CRS_VARIABLE_NAME}' not found in sample file.")
            ds_sample.close()
            print("--- End of sample file inspection ---")
            return True
    except Exception as e:
        print(f"Error opening/inspecting sample file {file_url}: {e}")
        return False

In [ ]:
inspect_sample_file(s3_file_urls[-1], s3)


--- Inspecting sample file: s3://noaa-nwm-retrospective-2-1-pds/model_output/2003/200301020000.LDASOUT_DOMAIN1.comp ---
Sample file opened successfully.

Variables:
- time (dims: ('time',), attrs: {'long_name': 'valid output time', 'standard_name': 'time', 'valid_min': np.int32(17356500), 'valid_max': np.int32(17485920)})
- reference_time (dims: ('reference_time',), attrs: {'long_name': 'model initialization time', 'standard_name': 'forecast_reference_time'})
- x (dims: ('x',), attrs: {'resolution': np.float32(1000.0), 'standard_name': 'projection_x_coordinate', 'long_name': 'x coordinate of projection', 'units': 'm', '_CoordinateAxisType': 'GeoX'})
- y (dims: ('y',), attrs: {'resolution': np.float32(1000.0), 'standard_name': 'projection_y_coordinate', 'long_name': 'y coordinate of projection', 'units': 'm', '_CoordinateAxisType': 'GeoY'})
- crs (dims: (), attrs: {'longitude_of_prime_meridian': np.float32(0.0), 'standard_parallel': array([30., 60.], dtype=float32), 'longitude_of_centr

True

In [ ]:
storage_options_s3 = {'anon': True, 'client_kwargs': {'region_name': 'us-east-1'}}
try:
    ds_nwm_full = xr.open_mfdataset(
        list(s3_file_urls),
        engine='h5netcdf',
        combine='by_coords',
        storage_options=storage_options_s3
        # parallel=True, # Optional
        #chunks={'y': 5, 'x': 5} # Adjust. Chunking after loading is also an option
    )
    print("Dataset opened successfully.")
except Exception as e:
    print(f"Error opening datasets: {e}")
    exit()

#ds_nwm_full = ds_nwm_full.chunk({'time': -1, 'y': 5, 'x': 5})
# 9 timestamps approximately takes 2  minutes to retrive and uses 2.5 GBs of RAM

Dataset opened successfully.


# Reproject point lat/long

In [ ]:
print("\nExtracting CRS information and transforming target Lat/Lon...")
nwm_crs = None
try:
    if CRS_VARIABLE_NAME not in ds_nwm_full:
        raise ValueError(f"CRS variable '{CRS_VARIABLE_NAME}' not found in the dataset. Cannot determine projection.")

    crs_attrs = ds_nwm_full[CRS_VARIABLE_NAME].attrs
    if not crs_attrs:
        raise ValueError(f"CRS variable '{CRS_VARIABLE_NAME}' has no attributes. Cannot determine projection.")

    # Attempt to parse CRS using pyproj.CRS.from_cf()
    # This is the preferred method if CF attributes are well-defined.
    print(f"Attempting to parse CRS from '{CRS_VARIABLE_NAME}' attributes using pyproj.CRS.from_cf...")
    try:
        nwm_crs = pyproj.CRS.from_cf(crs_attrs)
        print(f"Successfully parsed NWM CRS using from_cf: {nwm_crs.to_wkt(pretty=True)}")
    except Exception as e_crs_cf:
        print(f"Warning: pyproj.CRS.from_cf failed: {e_crs_cf}")
        print("Attempting to build Proj string manually for 'lambert_conformal_conic'.")
        print("PLEASE VERIFY these parameters against the output of 'inspect_sample_file'.")

        # --- MANUAL CRS CONSTRUCTION FOR LAMBERT CONFORMAL CONIC ---
        # Ensure these attribute names match what you see in `inspect_sample_file` output
        # for your 'crs' variable.
        # Required for LCC:
        # - standard_parallel (can be one or two values)
        # - longitude_of_central_meridian
        # - latitude_of_projection_origin
        # Optional but important for accuracy:
        # - false_easting
        # - false_northing
        # Earth model:
        # - semi_major_axis and semi_minor_axis (or inverse_flattening) for ellipsoidal
        # - OR sphere_radius (or semi_major_axis if inverse_flattening is 0 or not present) for spherical

        proj_params = {'proj': 'lcc'} # grid_mapping_name is lambert_conformal_conic

        # Standard Parallels (lat_1, lat_2 or lat_ts)
        sp = crs_attrs.get('standard_parallel')
        if sp is not None:
            if isinstance(sp, (list, tuple)) and len(sp) == 2:
                proj_params['lat_1'] = sp[0]
                proj_params['lat_2'] = sp[1]
            elif isinstance(sp, (list, tuple)) and len(sp) == 1:
                 proj_params['lat_1'] = sp[0] # Or sometimes 'lat_ts' for single standard parallel
                 # proj_params['lat_ts'] = sp[0] # Some LCC variants use lat_ts for one std parallel
            elif isinstance(sp, (int, float)):
                proj_params['lat_1'] = sp
                # proj_params['lat_ts'] = sp
            else:
                print("Warning: 'standard_parallel' attribute has unexpected format. Using defaults.")
                proj_params['lat_1'] = 30.0 # Default, VERIFY
                proj_params['lat_2'] = 60.0 # Default, VERIFY
        else:
            print("Warning: 'standard_parallel' not found. Using defaults.")
            proj_params['lat_1'] = 30.0 # Default, VERIFY
            proj_params['lat_2'] = 60.0 # Default, VERIFY


        proj_params['lon_0'] = crs_attrs.get('longitude_of_central_meridian', -97.0) # VERIFY
        proj_params['lat_0'] = crs_attrs.get('latitude_of_projection_origin', 40.0) # VERIFY
        proj_params['x_0'] = crs_attrs.get('false_easting', 0.0) # VERIFY
        proj_params['y_0'] = crs_attrs.get('false_northing', 0.0) # VERIFY

        # Earth Model (Ellipsoid or Sphere)
        # Check for spherical first (e.g., inverse_flattening is 0 or semi_major == semi_minor)
        inv_flat = crs_attrs.get('inverse_flattening')
        semi_major = crs_attrs.get('semi_major_axis')
        semi_minor = crs_attrs.get('semi_minor_axis')
        sphere_radius = crs_attrs.get('sphere_radius') # Less common CF name, but possible

        if sphere_radius is not None:
            proj_params['R'] = sphere_radius
            print(f"Using spherical earth model with R={sphere_radius} from 'sphere_radius'.")
        elif semi_major is not None and (inv_flat == 0.0 or (semi_minor is not None and semi_major == semi_minor)):
            proj_params['R'] = semi_major
            print(f"Using spherical earth model with R={semi_major} from 'semi_major_axis' (spherical implied).")
        elif semi_major is not None:
            proj_params['a'] = semi_major
            if inv_flat is not None and inv_flat != 0: # Check inv_flat is not None before comparing
                proj_params['rf'] = inv_flat # rf is 1/f, so if inv_flat is f, it should be 1/inv_flat. Pyproj takes f.
                                             # CF 'inverse_flattening' IS f. So this is correct.
            elif semi_minor is not None:
                proj_params['b'] = semi_minor
            else: # Default to WGS84 if only semi_major is given for an ellipsoid
                print("Warning: Ellipsoidal parameters incomplete, defaulting to WGS84 ellipsoid for Proj.")
                proj_params['ellps'] = 'WGS84'
            print(f"Using ellipsoidal earth model. a={proj_params.get('a')}, rf={proj_params.get('rf')}, b={proj_params.get('b')}")
        else:
            print("Warning: Earth model parameters (semi_major_axis, etc.) not found. Defaulting to WGS84 ellipsoid for Proj.")
            proj_params['ellps'] = 'WGS84' # Default ellipsoid

        proj_params['units'] = 'm' # Usually meters for NWM

        nwm_crs = pyproj.CRS.from_dict(proj_params)
        print(f"NWM CRS constructed manually: {nwm_crs.to_wkt(pretty=True)}")

    if nwm_crs is None:
        raise ValueError("Failed to determine NWM CRS from both from_cf and manual construction.")

    # Define WGS84 CRS (standard lat/lon)
    wgs84_crs = pyproj.CRS("EPSG:4326")

    # Create a transformer
    transformer = pyproj.Transformer.from_crs(wgs84_crs, nwm_crs, always_xy=True)

    # Transform target lat/lon to NWM's x/y
    target_x, target_y = transformer.transform(TARGET_LON, TARGET_LAT)
    print(f"Target Lat/Lon ({TARGET_LAT}, {TARGET_LON}) transformed to NWM x/y: ({target_x:.2f}, {target_y:.2f})")

except Exception as e:
    print(f"Error during CRS processing or coordinate transformation: {e}")
    print("Please ensure the CRS_VARIABLE_NAME is correct and its attributes define a recognizable projection.")
    print("Run the 'inspect_sample_file' block to see the CRS attributes and adjust manual construction if needed.")
    exit()


Extracting CRS information and transforming target Lat/Lon...
Attempting to parse CRS from 'crs' attributes using pyproj.CRS.from_cf...
Successfully parsed NWM CRS using from_cf: PROJCRS["Lambert_Conformal_Conic",
    BASEGEOGCRS["Unknown datum based upon the Authalic Sphere",
        DATUM["Not specified (based on Authalic Sphere)",
            ELLIPSOID["Sphere",6370000,0,
                LENGTHUNIT["metre",1]],
            ID["EPSG",6035]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["Degree",0.0174532925199433]]],
    CONVERSION["unnamed",
        METHOD["Lambert Conic Conformal (2SP)",
            ID["EPSG",9802]],
        PARAMETER["Easting at false origin",0,
            LENGTHUNIT["metre",1],
            ID["EPSG",8826]],
        PARAMETER["Northing at false origin",0,
            LENGTHUNIT["metre",1],
            ID["EPSG",8827]],
        PARAMETER["Longitude of false origin",-97,
            ANGLEUNIT["Degree",0.0174532925199433],
            ID["EPSG",8822]],
     

#Subsetting variable(s)

In [ ]:
TARGET_VARIABLES = ['UGDRNOFF'] # REPLACE for variable of ineterest

In [ ]:
print(f"\nSelecting variables: {TARGET_VARIABLES}...")
try:
    missing_vars = [var for var in TARGET_VARIABLES if var not in ds_nwm_full.data_vars]
    if missing_vars:
        raise KeyError(f"Target variables not found: {missing_vars}. Available: {list(ds_nwm_full.data_vars)}")
    ds_selected_vars = ds_nwm_full[TARGET_VARIABLES]
except KeyError as e:
    print(f"Error selecting variables: {e}")
    exit()


Selecting variables: ['UGDRNOFF']...


In [ ]:
ds_selected_vars

<xarray.Dataset> Size: 3GB
Dimensions:         (reference_time: 2, time: 9, y: 3840, x: 4608)
Coordinates:
  * time            (time) datetime64[ns] 72B 2003-01-01 ... 2003-01-02
  * x               (x) float64 37kB -2.303e+06 -2.302e+06 ... 2.304e+06
  * y               (y) float64 31kB -1.92e+06 -1.919e+06 ... 1.919e+06
  * reference_time  (reference_time) datetime64[ns] 16B 2002-10-01 2003-01-01
Data variables:
    UGDRNOFF        (reference_time, time, y, x) float64 3GB dask.array<chunksize=(1, 1, 768, 922), meta=np.ndarray>
Attributes:
    TITLE:                      OUTPUT FROM WRF-Hydro v5.2.0-beta2
    model_initialization_time:  2002-10-01_00:00:00
    model_output_valid_time:    2003-01-01_00:00:00
    model_total_valid_times:    736
    Conventions:                CF-1.6
    code_version:               v5.2.0-beta2
    model_output_type:          land
    model_configuration:        retrospective
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    GDAL_DataType:              Generic

In [ ]:
data_at_location = ds_selected_vars[TARGET_VARIABLES[0]].sel(
    y=target_y,
    x=target_x,
    reference_time=ds_selected_vars.reference_time[1],
    method="nearest",  # Finds closest grid cell
)
# Output: DataArray with shape (time,) containing values at the nearest (lat, lon)
print(data_at_location)

<xarray.DataArray 'UGDRNOFF' (time: 9)> Size: 72B
dask.array<getitem, shape=(9,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time            (time) datetime64[ns] 72B 2003-01-01 ... 2003-01-02
    x               float64 8B 5.245e+05
    y               float64 8B 5.345e+05
    reference_time  datetime64[ns] 8B 2003-01-01
Attributes:
    long_name:       Accumulated underground runoff
    units:           mm
    grid_mapping:    crs
    valid_range:     [  -10000 10000000]
    esri_pe_string:  PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DAT...


In [ ]:
df = pd.DataFrame({
    "time":data_at_location.time.values,           # Timestamps
    TARGET_VARIABLES[0]: data_at_location.values,         # Extracted values
    "y": data_at_location.y.values,             # Nearest lat (optional)
    "x": data_at_location.x.values,             # Nearest lon (optional)
})
print(df)

                 time  UGDRNOFF            y            x
0 2003-01-01 00:00:00       NaN  534499.6875  524500.8125
1 2003-01-01 03:00:00      0.01  534499.6875  524500.8125
2 2003-01-01 06:00:00      0.03  534499.6875  524500.8125
3 2003-01-01 09:00:00      0.04  534499.6875  524500.8125
4 2003-01-01 12:00:00      0.05  534499.6875  524500.8125
5 2003-01-01 15:00:00      0.06  534499.6875  524500.8125
6 2003-01-01 18:00:00      0.08  534499.6875  524500.8125
7 2003-01-01 21:00:00      0.09  534499.6875  524500.8125
8 2003-01-02 00:00:00      0.10  534499.6875  524500.8125
